<a href="https://colab.research.google.com/github/oksami/cs6320project/blob/main/Protege_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install owlready2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 45.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.47-cp310-cp310-linux_x86_64.whl size=24075198 sha256=5fdcad18c2b815e5211044fce43f634aa507507662a7ec0958ba9435813e328a
  Stored in directory: /root/.cache/pip/wheels/27/3e/ba/4171c4b10bba9fe1774fbf8fcf794de889e636ce64ad83a533
Successfully built owlready2


In [3]:
import torch
import re
import pandas as pd
from owlready2 import *

onto = get_ontology("/content/drive/MyDrive/Data_NLP/Protege_project.rdf").load()
# Map recognized fund types to ontology class names
fund_type_mapping = {
    "fossil-free": "FossilFreeFund",
    "deforestation-free": "DeforestationFreeFund",
    "gender equality": "GenderEqualityFund",
    "gun-free": "GunFreeFund",
    "prison-free": "PrisonFreeFund",
    "tobacco-free": "TobaccoFreeFund",
    "weapon-free": "WeaponFreeFund"
}

# Query ontology function (updated for multiple grades)
def query_ontology(onto, fund_types, grades):
    results = defaultdict(list)  # Group results by grade
    for fund_type in fund_types:
        fund_class = getattr(onto, fund_type_mapping[fund_type], None)
        if fund_class:
            for individual in fund_class.instances():
                if hasattr(individual, "Grade"):
                    # Check each grade in the query and group funds by grade
                    for grade in grades:
                        if grade in individual.Grade:
                            results[grade].append({
                                "name": individual.name,
                                "grade": grade,
                                "share_class": getattr(individual, "ShareClassType", "N/A"),
                                "category": getattr(individual, "CategoryGroup", "N/A"),
                                "weight": getattr(individual, "Weight", "N/A")
                            })

    return results

# Display results in a chatbot-friendly format (updated for multiple grades)
def display_detailed_results(funds_by_grade, grades, fund_types):
    if not funds_by_grade:
        return f"I'm sorry, I couldn't find any funds matching {(fund_types)} with grades: {(grades)}."

    response = f"Here are the top recommendations for {(fund_types)} funds with the requested grades:\n\n"

    for grade in grades:
        if grade in funds_by_grade:
            response += f"Grade: {grade}\n"
            for idx, fund in enumerate(funds_by_grade[grade][:5], 1):  # Limit to top 5 per grade
                response += (
                    f"{idx}. Fund: {fund['name']}\n"
                    f"   - Share Class: {fund['share_class']}\n"
                    f"   - Category: {fund['category']}\n"
                    f"   - Weight: {fund['weight']}\n"
                )
            response += "\n"

    response += "Would you like more funds under any other criteria?"
    return response
